In [1]:
from ragcar import Ragcar
from ragcar.utils import PromptTemplate

/home/pcn_rns/anaconda3/envs/rc_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = Ragcar(
    tool="text_generation", 
    src="sllm", 
    model="upstage/SOLAR-10.7B-Instruct-v1.0", 
    prompt_template=PromptTemplate("{input} 수도는?"), 
    lora_model_dir="/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13", 
    adapter="lora",
    stream=True,
    use_async=True
    )

INFO:text_generation:Loading SLLM model
INFO:sllm:model_n: upstage/SOLAR-10.7B-Instruct-v1.0, lora_path: /home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13,  adapter: lora, formatting: False
INFO:ragcar.models.base:Using device cuda with dtype torch.float16
INFO:ragcar.models.base:Using device cuda with dtype torch.float16
INFO:ragcar.models.base:patching _expand_mask in llama models
INFO:ragcar.models.base:model_kwargs: {'torch_dtype': torch.float16, 'load_in_8bit': False, 'load_in_4bit': False, 'device_map': 'auto'}
INFO:ragcar.models.base:loading llama model


EOS: 2 / </s>
BOS: 1 / <s>
PAD: 2 / </s>
UNK: 0 / <unk>


Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.59it/s]
INFO:ragcar.models.base:resizing token embeddings to 32000
INFO:ragcar.models.base:model config: cuda
INFO:ragcar.models.base:GPU memory usage after model load: 4.735GB (+0.045GB cache, +53.409GB misc)
INFO:ragcar.models.base:converting modules to torch.float16 for flash attention
INFO:ragcar.models.base:1. Loading pretrained PEFT - lora
INFO:ragcar.models.base:GPU memory usage after adapters: 4.788GB (+0.766GB cache, +53.409GB misc)
INFO:text_generation:Loaded SLLM model: <ragcar.models.sllm.SLLMCompletion object at 0x7f3d0bc05eb0>
INFO:text_generation:RagcarTextGeneration: <ragcar.models.sllm.SLLMCompletion object at 0x7f3d0bc05eb0>, <ragcar.utils.prompt_template.PromptTemplate object at 0x7f3d0bbdddf0>, {'max_tokens': 1000, 'temperature': 0.1, 'top_p': 1.0, 'do_sample': True, 'use_cache': True, 'return_dict_in_generate': True, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False}


trainable params: 0 || all params: 10,857,353,216 || trainable%: 0.0


In [3]:
generator(input="프량스", temperature=0.01)

INFO:text_generation:formatted_prompt: 프량스 수도는?
INFO:sllm:device: cuda
INFO:sllm:messages: 프량스 수도는?
INFO:sllm:model: PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embe

In [7]:
import asyncio

async def main():
    generator = Ragcar(
        tool="text_generation", 
        src="sllm", 
        model="upstage/SOLAR-10.7B-Instruct-v1.0", 
        prompt_template=PromptTemplate("{input} 수도는?"), 
        lora_model_dir="/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13", 
        adapter="lora",
        stream=True,
        use_async=True
    )

    # 비동기 메소드 호출
    async for output in generator(input="프량스", temperature=0.01):
        print(output)

# 이벤트 루프 실행
asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

/home/pcn_rns/anaconda3/envs/rc_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model= "upstage/SOLAR-10.7B-Instruct-v1.0" #"mistralai/Mistral-7B-v0.1"  #"upstage/SOLAR-10.7B-Instruct-v1.0" 
peft_model_name = '/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_15'
config = PeftConfig.from_pretrained(peft_model_name)
config.base_model_name_or_path = model
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  device_map="cuda:1")

model = PeftModel.from_pretrained(base_model, peft_model_name)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

Loading checkpoint shards: 100%|██████████| 5/5 [00:09<00:00,  1.85s/it]


In [3]:
def len_tokens(txt, model_name): 
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return len(tokenizer.encode(txt))

In [4]:
txt = """Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
다음주에 중구에서 하는 뮤지컬 공연 정보 알려줘

### Input:##API 호출 정의 
-culture_seoul_events: {"Name": "서울 문화행사 조회", "Description": "서울문화포털 API를 사용합니다. 콘서트, 뮤지컬, 무용, 클래식, 전시/미술, 국악, 축제, 교육체험 등의 정보를 제공합니다. It offers details such as title, location, date, organizing institution, target audience, fees, performers, Booking homepage address, and program for performances and events.", "OptionalKeys": {"location": "District name in Seoul you wish to inquire about.", "startdate": "Specific start date you wish to inquire about(format: %yyyy-mm-dd)", "enddate": "Specific end date you wish to inquire about(format: %yyyy-mm-dd)"}}
-weather_forecast: {"Name": "기상청 날씨 조회", "Description": "기상청 API허브를 사용합니다. It provides detailed weather information for specific regions and dates in South Korea. It offers data about temperature, humidity, rainfall, wind speed. Weather forecast offer weather information within 10 days of the current date.", "RequiredKeys": {"region": "The city or district in South Korea you wish to inquire about (e.g., '서울', '부산', '강남구', '동자동')", "datetime": "Specific date you wish to inquire about (format:%yyyymmdd)"}}
-search_google: {"Name": "구글 검색", "Description": "구글 검색 API를 사용합니다. The tool searches for information on the internet and blogs. The search results are provided in the form of URL, title, and content.(최대 100자까지)"}

**날짜데이터 
오늘: Wednesday, December 20, 2023
내일: Thursday, December 21, 2023
모레: Friday, December 22, 2023
이번주: Wednesday, December 20, 2023 ~ Sunday, December 24, 2023
다음주: Monday, December 25, 2023 ~ Sunday, December 31, 2023

**이전 대화 데이터
User는 12월 22일부터 말까지 하는 뮤지컬공연이랑 클래식 공연 정보 알려줘.
AI는 12월 22일부터 12월 말까지 서울에서 진행되는 뮤지컬과 클래식 공연 정보를 안내해드리겠습니다.. 사용자는 이 공연 장소의 주차장이 있어? AI는 네, 세종M씨어터와 세종체임버홀에서 진행되는 공연 장소인 세종문화회관에는 주차장이 마련되어 있습니다. 세종문화회관 공영주차장의 주차요금 및 요금할인 조건에 대한 정보는 세종문화회관 공영주차장 주차요금 및 요금할인 조건 또는 세종로 공영주차장 이용안내를 확인하시면 됩니다. 관람객 할인이 적용되는 경우도 있으니, 공연 관람 전에 해당 정보를 미리 확인하시는 것이 좋겠습니다.

[/INST]### Output:"""

output = """{"weather_forecast":[{"RequiredKeys":{"region":"용산구","datetime":"20231220"},"response":"날씨 정보 요청에 필요한 지역 정보가 누락되었습니다. 어떤 지역의 날씨를 알고 싶으신지 알려주세요."}]}</s>"""
len_tokens(output, "upstage/SOLAR-10.7B-Instruct-v1.0")

102

In [4]:
#8it quantization

# model="upstage/SOLAR-10.7B-Instruct-v1.0"
# peft_model_name = "/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13"
# config = PeftConfig.from_pretrained(peft_model_name)
# config.base_model_name_or_path = model
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True
# )
# # base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  device_map="cuda:2")
# base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="cuda:0")
 
# model = PeftModel.from_pretrained(base_model, peft_model_name)
 
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# streamer = TextStreamer(tokenizer)

In [5]:
tool_1 = """culture_seoul_events: {"Name": "서울 문화행사 조회", "Description": "서울문화포털 API를 사용합니다. 콘서트, 뮤지컬, 무용, 클래식, 전시/미술, 국악, 축제, 교육체험 등의 정보를 제공합니다. It offers details such as title, location, date, organizing institution, target audience, fees, performers, Booking homepage address, and program for performances and events.", "OptionalKeys": {"location": "District name in Seoul you wish to inquire about.", "startdate": "Specific start date you wish to inquire about(format: %yyyy-mm-dd)", "enddate": "Specific end date you wish to inquire about(format: %yyyy-mm-dd)"}}"""
tool_2 = """weather_forecast: {"Name": "기상청 날씨 조회", "Description": "기상청 API허브를 사용합니다. It provides detailed weather information for specific regions and dates in South Korea. It offers data about temperature, humidity, rainfall, wind speed. Weather forecast offer weather information within 10 days of the current date.", "RequiredKeys": {"region": "The city or district in South Korea you wish to inquire about (e.g., '서울', '부산', '강남구', '동자동')", "datetime": "Specific date you wish to inquire about (format:%yyyymmdd)"}}"""
tool_3 = """search_google: {"Name": "구글 검색", "Description": "구글 검색 API를 사용합니다. The tool searches for information on the internet and blogs. The search results are provided in the form of URL, title, and content.(최대 100자까지)"}"""
date = "Tuesday, November 28, 2023"
history = "User는 12월 22일부터 말까지 하는 뮤지컬공연이랑 클래식 공연 정보 알려줘.\nAI는 12월 22일부터 12월 말까지 서울에서 진행되는 뮤지컬과 클래식 공연 정보를 안내해드리겠습니다."
user = "이 공연 장소의 주차장이 있어?"
ai = "네, 세종M씨어터와 세종체임버홀에서 진행되는 공연 장소인 세종문화회관에는 주차장이 마련되어 있습니다. 세종문화회관 공영주차장의 주차요금 및 요금할인 조건에 대한 정보는 세종문화회관 공영주차장 주차요금 및 요금할인 조건 또는 세종로 공영주차장 이용안내를 확인하시면 됩니다. 관람객 할인이 적용되는 경우도 있으니, 공연 관람 전에 해당 정보를 미리 확인하시는 것이 좋겠습니다."
system_prompt = f"""Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야."""

user_prompt = f"""##API 호출 정의 
-{tool_1}
-{tool_2}
-{tool_3}

**날짜데이터 
오늘: Wednesday, December 20, 2023
내일: Thursday, December 21, 2023
모레: Friday, December 22, 2023
이번주: Wednesday, December 20, 2023 ~ Sunday, December 24, 2023
다음주: Monday, December 25, 2023 ~ Sunday, December 31, 2023

**이전 대화 데이터
{history}. 사용자는 {user} AI는 {ai}

"""

def gen(x):
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.95,
        top_k=200,
        max_new_tokens=300,
        # early_stopping=True,
        do_sample=True,
        # num_beams=3,
        # no_repeat_ngram_size=5,
    )
    q = f"{system_prompt}\n\n[INST]### Instruction:\n{x}\n\n### Input:{user_prompt}\n\n[/INST]### Output: "
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda:1'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)
    print("start_index", start_index)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [6]:
import time
start_time = time.time()
gen('오늘의 용산구 날씨도 같이 알려줘')
response_time = time.time() - start_time
print(response_time)

<s> Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
오늘의 용산구 날씨도 같이 알려줘

### Input:##API 호출 정의 
-culture_seoul_events: {"Name": "서울 문화행사 조회", "Description": "서울문화포털 API를 사용합니다. 콘서트, 뮤지컬, 무용, 클래식, 전시/미술, 국악, 축제, 교육체험 등의 정보를 제공합니다. It offers details such as title, location, date, organizing institution, target audience, fees, performers, Booking homepage address, and program for performances and events.", "OptionalKeys": {"location": "District name in Seoul you wish to inquire about.", "startdate": "Specific start date you wish to inquire about(format: %yyyy-mm-dd)", "enddate": "Specific end date you wish to inquire about(format: %yyyy-mm-dd)"}}
-weather_forecast: {"Name": "기상청 날씨 조회", "Description": "기상청 API허브를 사용합니다. It provides detailed weather information for specific regions and dates in South Korea. It offers data about temperature, humidity, rainfall, wind speed. Weather forecast of

 {"weather_forecast":[{"RequiredKeys":{"region":"용산구","datetime":"20231220"},"response":"날씨 정보 요청에 필요한 지역 정보가 누락되었습니다. 어떤 지역의 날씨를 알고 싶으신지 알려주세요."}]}</s>
start_index 2060
14.624669551849365


In [6]:
response_time

28.062216997146606

## axolotl inference 코드만 작성해서 속도 측정하기

In [1]:
import logging
import math
import os
from typing import Any, Dict, Optional, Tuple, Union  # noqa: F401

import addict
import bitsandbytes as bnb
import torch
import transformers
from peft import (
    LoftQConfig,
    PeftConfig,
    PeftModel,
    PeftModelForCausalLM,
    prepare_model_for_kbit_training,
)
from peft.tuners.lora import QuantLinear
from transformers import (  # noqa: F401
    AddedToken,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GPTQConfig,
    PreTrainedModel,
    PreTrainedTokenizerBase,
)
from transformers.integrations.deepspeed import is_deepspeed_zero3_enabled

/home/pcn_rns/anaconda3/envs/rc_test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from addict import Dict


class DictDefault(Dict):
    """
    A Dict that returns None instead of returning empty Dict for missing keys.
    """

    def __missing__(self, key):
        return None

    def __or__(self, other):
        return DictDefault(super().__or__(other))

In [3]:
from types import SimpleNamespace
#solar.yml 파일을 딕셔너리로 변환
# cfg_dict = {'base_model': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_type': 'LlamaForCausalLM', 'tokenizer_type': 'LlamaTokenizer', 'is_llama_derived_model': True, 'load_in_8bit': False, 'load_in_4bit': False, 'strict': False, 'sequence_len': 4096, 'sample_packing': False, 'pad_to_sequence_len': True, 'adapter': 'lora', 'lora_model_dir': '/home/pcn_rns/ella/git_frameworks/axolotl/output_solor/exp_13', 'lora_r': 32, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_target_linear': True, 'lora_fan_in_fan_out': None, 'gradient_accumulation_steps': 8, 'micro_batch_size': 8, 'num_epochs': 4, 'optimizer': 'adamw_bnb_8bit', 'lr_scheduler': 'cosine', 'learning_rate': 1e-05, 'train_on_inputs': False, 'group_by_length': False, 'bf16': True, 'fp16': False, 'tf32': False, 'gradient_checkpointing': True, 'early_stopping_patience': None, 'resume_from_checkpoint': None, 'local_rank': 0, 'logging_steps': 1, 'xformers_attention': True, 'flash_attention': False, 's2_attention': None, 'warmup_steps': 10, 'evals_per_epoch': 4, 'eval_table_size': 0, 'eval_max_new_tokens': 500, 'saves_per_epoch': 1, 'debug': None, 'deepspeed': 'deepspeed_configs/zero1.json', 'weight_decay': 0.0, 'fsdp': None, 'fsdp_config': None, 'special_tokens': None, 'batch_size': 64, 'eval_batch_size': 8, 'world_size': 1, 'eval_causal_lm_metrics': ['sacrebleu', 'comet', 'ter', 'chrf'], 'device': 'cuda:0', 'device_map': None, 'ddp': False, 'torch_dtype': torch.bfloat16, 'save_steps': 0.25, 'eval_steps': 0.0625, 'dataset_processes': 128, 'base_model_config': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_config_type': 'llama', 'is_falcon_derived_model': False, 'is_mistral_derived_model': False, 'is_qwen_derived_model': None, 'gradient_checkpointing_kwargs': {'use_reentrant': True}, 'use_wandb': True}

#solar_qlora
cfg_dict = {'base_model': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_type': 'LlamaForCausalLM', 'tokenizer_type': 'LlamaTokenizer', 'is_llama_derived_model': True, 'load_in_8bit': False, 'load_in_4bit': True, 'strict': False,  'adapter': 'qlora', 'sample_packing': False, 'pad_to_sequence_len': True,  'lora_target_linear': True, 'lora_fan_in_fan_out': None, 'xformers_attention': None, 'flash_attention': True, 'world_size': 1, 'eval_max_new_tokens': 128, 'eval_causal_lm_metrics': ['sacrebleu', 'comet', 'ter', 'chrf'], 'device': 'cuda:0', 'device_map': None, 'ddp': False, 'torch_dtype': torch.bfloat16, 'save_steps': 0.14285714285714285, 'eval_steps': 0.03571428571428571, 'dataset_processes': 128, 'base_model_config': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_config_type': 'llama', 'is_falcon_derived_model': False, 'is_mistral_derived_model': False, 'is_qwen_derived_model': None, 'gradient_checkpointing_kwargs': {'use_reentrant': True}, 'use_wandb': True}
# cfg_dict = {'base_model': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_type': 'LlamaForCausalLM', 'tokenizer_type': 'LlamaTokenizer', 'is_llama_derived_model': True, 'load_in_8bit': False, 'load_in_4bit': True, 'strict': False, 'datasets': [{'path': 'datasets_cleansinng/datasets/helper_selector_1280_0305_v01.jsonl', 'type': {'system_prompt': 'Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.', 'format': '[INST]### Instruction:\n{instruction}\n\n### Input:{input}\n\n[/INST]### Output: ', 'no_input_format': '[INST]### Instruction:\n{instruction}\n\n[/INST]### Output: ', 'field_instruction': 'Instruction', 'field_input': 'Input', 'field_output': 'Output'}}], 'dataset_prepared_path': None, 'val_set_size': 0.1, 'adapter': 'qlora', 'lora_model_dir': './output_solor/exp_15', 'sequence_len': 4096, 'sample_packing': False, 'pad_to_sequence_len': True, 'lora_r': 32, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_target_modules': None, 'lora_target_linear': True, 'lora_fan_in_fan_out': None, 'wandb_project': 'deepspeed_mistral', 'wandb_entity': None, 'wandb_watch': None, 'wandb_name': 'exp_15', 'wandb_log_model': None, 'gradient_accumulation_steps': 8, 'micro_batch_size': 8, 'num_epochs': 7, 'optimizer': 'adamw_bnb_8bit', 'lr_scheduler': 'cosine', 'learning_rate': 0.0005, 'train_on_inputs': False, 'group_by_length': False, 'bf16': True, 'fp16': False, 'tf32': False, 'gradient_checkpointing': True, 'early_stopping_patience': None, 'resume_from_checkpoint': None, 'local_rank': 0, 'logging_steps': 1, 'xformers_attention': None, 'flash_attention': True, 'warmup_steps': 10, 'evals_per_epoch': 4, 'eval_table_size': 0, 'saves_per_epoch': 1, 'debug': None, 'deepspeed': 'deepspeed_configs/zero1.json', 'weight_decay': 0.0, 'fsdp': None, 'fsdp_config': None, 'special_tokens': None, 'axolotl_config_path': 'examples/llama-2/qlora-deepspeed_solar.yml', 'batch_size': 64, 'eval_batch_size': 8, 'world_size': 1, 'eval_max_new_tokens': 128, 'eval_causal_lm_metrics': ['sacrebleu', 'comet', 'ter', 'chrf'], 'device': 'cuda:0', 'device_map': None, 'ddp': False, 'torch_dtype': torch.bfloat16, 'save_steps': 0.14285714285714285, 'eval_steps': 0.03571428571428571, 'dataset_processes': 128, 'base_model_config': 'upstage/SOLAR-10.7B-Instruct-v1.0', 'model_config_type': 'llama', 'is_falcon_derived_model': False, 'is_mistral_derived_model': False, 'is_qwen_derived_model': None, 'gradient_checkpointing_kwargs': {'use_reentrant': True}, 'use_wandb': True}


# mistral 
# cfg_dict = {'base_model': 'mistralai/Mistral-7B-v0.1', 'model_type': 'MistralForCausalLM', 'tokenizer_type': 'LlamaTokenizer', 'is_mistral_derived_model': True, 'load_in_8bit': False, 'load_in_4bit': False, 'strict': False,  'val_set_size': 0.05, 'adapter': 'lora', 'lora_r': 64, 'lora_alpha': 64, 'lora_dropout': 0.05, 'lora_target_modules': ['q_proj', 'v_proj'], 'sequence_len': 4096, 'sample_packing': False, 'pad_to_sequence_len': True, 'gradient_accumulation_steps': 8, 'micro_batch_size': 16, 'num_epochs': 5, 'optimizer': 'adamw_bnb_8bit', 'lr_scheduler': 'cosine', 'learning_rate': 5e-05, 'train_on_inputs': False, 'group_by_length': False, 'bf16': True, 'fp16': False, 'tf32': False, 'gradient_checkpointing': True, 'early_stopping_patience': None, 'resume_from_checkpoint': None, 'local_rank': 0, 'logging_steps': 1, 'xformers_attention': None, 'flash_attention': True, 'warmup_steps': 10, 'evals_per_epoch': 5, 'eval_table_size': 0, 'eval_max_new_tokens': 128, 'saves_per_epoch': 1, 'debug': None, 'deepspeed': 'deepspeed_configs/zero1.json', 'weight_decay': 0.0, 'fsdp': None, 'fsdp_config': None, 'special_tokens': {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, 'axolotl_config_path': 'examples/mistral/Mistral-7b-example/lora_v2.yml', 'lora_model_dir': '/home/pcn_rns/ella/git_frameworks/axolotl/mistral_output/exp_2', 'batch_size': 128, 'eval_batch_size': 16, 'world_size': 1, 'eval_causal_lm_metrics': ['sacrebleu', 'comet', 'ter', 'chrf'], 'device': 'cuda:0', 'device_map': None, 'ddp': False, 'torch_dtype': torch.bfloat16, 'save_steps': 0.2, 'eval_steps': 0.04, 'dataset_processes': 128, 'base_model_config': 'mistralai/Mistral-7B-v0.1', 'model_config_type': 'mistral', 'is_llama_derived_model': False, 'is_falcon_derived_model': False, 'is_qwen_derived_model': None, 'gradient_checkpointing_kwargs': {'use_reentrant': True}, 'use_wandb': True}
# 딕셔너리를 SimpleNamespace 객체로 변환
cfg = DictDefault(**cfg_dict)

 

In [4]:
if cfg.lora_target_linear:
    print("hi")

hi


In [5]:
# cfg.trust_remote_code = False
# cfg.gpu_memory_limit =None
# cfg.max_memory = None
# cfg.resize_token_embeddings_to_32x = None
# cfg.lora_target_modules= None
# cfg.peft_layers_to_transform = None
# cfg.peft_layers_to_transform =None
# cfg.lora_modules_to_save =None


# #mistral 추가
# cfg.s2_attention = None
# cfg.lora_target_linear = None

In [6]:
def get_linear_embedding_layers(model_type):
    """
    returns the linear embedding layers needed for loras, dependent on the model arch
    """
    if model_type == "gpt_neox":
        return ["embed_in", "embed_out"]
    if model_type == "falcon":
        return ["word_embeddings", "lm_head"]
    return ["embed_tokens", "lm_head"]

# def check_model_config(cfg,  model_config):
#     quant_config_exists = (
#         hasattr(model_config, "quantization_config")
#         and model_config.quantization_config
#     )

#     lora_modules_to_save = get_linear_embedding_layers(model_config.model_type)


# load_model_config 함수에 로그 추가
def load_model_config(cfg):
    model_config_name = cfg.base_model_config or cfg.base_model
    # if not model_config_name and cfg.tokenizer_config:
    #     model_config_name = cfg.tokenizer_config
    trust_remote_code = cfg.trust_remote_code is True
    config_kwargs = {}

    try:
        model_config = AutoConfig.from_pretrained(
            model_config_name,
            trust_remote_code=trust_remote_code,
            **config_kwargs,
        )
    except ValueError as err:
        raise err
    
    return model_config



def load_tokenizer(cfg):
    model_config = load_model_config(cfg)
    tokenizer_kwargs = {}
    use_fast = True  # this is the default

    tokenizer_cls = AutoTokenizer
    if cfg.tokenizer_type:
        tokenizer_cls = getattr(transformers, cfg.tokenizer_type)

    tokenizer_config = cfg.base_model_config or cfg.base_model # tokenizer_config
    tokenizer = tokenizer_cls.from_pretrained(
        tokenizer_config,
        trust_remote_code=cfg.trust_remote_code or False,
        use_fast=use_fast,
        **tokenizer_kwargs,
    )

    if (
        tokenizer.__class__.__name__
        in [
            "LlamaTokenizer",
            "LlamaTokenizerFast",
            "CodeLlamaTokenizer",
            "CodeLlamaTokenizerFast",
        ]
        and hasattr(tokenizer, "pad_token")
        and not tokenizer.pad_token
    ):
         
        # set a pad_token, but use eos_token so we don't add a new token
        tokenizer.pad_token = "</s>"

    if tokenizer.__class__.__name__ == "GPTNeoXTokenizerFast":
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})
        os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Mistral's official FA implementation requires left padding
    if cfg.is_mistral_derived_model and cfg.flash_attention and not cfg.sample_packing:
        tokenizer.padding_side = "left"

    additional_special_tokens = None
    if cfg.special_tokens:
        print(cfg.special_tokens)
        special_tokens = cfg.special_tokens
        additional_special_tokens = special_tokens.pop(
            "additional_special_tokens", None
        )
        lora_modules_to_save = get_linear_embedding_layers(model_config.model_type)
        for k, val in special_tokens.items():
            # check if new special token is not already in tokenizer and
            # is adapter training to make sure lora_modules_to_save is set
            # pylint: disable=too-many-boolean-expressions
            if (
                (getattr(tokenizer, k) is None or getattr(tokenizer, k) != val)
                and (len(tokenizer.encode(val, add_special_tokens=False)) > 2)
                and cfg.adapter
                and (
                    not cfg.lora_modules_to_save
                    or not all(
                        x in cfg.lora_modules_to_save for x in lora_modules_to_save
                    )
                )
            ):
                 
                lora_modules_to_save = ", ".join(
                    [f"`{x}`" for x in lora_modules_to_save]
                )
                raise ValueError(
                    f"Please set lora_modules_to_save to [{lora_modules_to_save}] when using an adapter and changing the special tokens."
                )
            tokenizer.add_special_tokens(
                {k: AddedToken(val, rstrip=False, lstrip=False, normalized=False)}
            )

        bos_or_eos_in_special_tokens = (
            "bos_token" in cfg.special_tokens and "eos_token" in cfg.special_tokens
        )
        if (
            tokenizer.__class__.__name__
            in (
                "LlamaTokenizerFast",
                "CodeLlamaTokenizerFast",
            )
            and bos_or_eos_in_special_tokens
        ):
            tokenizer.update_post_processor()

    return tokenizer

In [7]:
"""
Shared utils for the monkeypatches
"""
from typing import Optional

import torch
import torch.nn.functional as F
from transformers.modeling_attn_mask_utils import (
    _prepare_4d_causal_attention_mask,
    _prepare_4d_causal_attention_mask_for_sdpa,
)
from transformers.utils import is_torch_bf16_gpu_available


@torch.jit.script
def get_max_seqlen_in_batch(attention_mask: torch.Tensor) -> torch.Tensor:
    max_num = int(torch.max(attention_mask).item())
    batch_size, _ = attention_mask.shape
    counts = torch.zeros((batch_size, max_num), dtype=torch.int32)

    for i in range(1, max_num + 1):
        mask = attention_mask == i
        counts[:, i - 1] = torch.sum(mask, dim=-1).to(dtype=torch.int32)

    result = counts.flatten()
    nonzero_indices = torch.nonzero(result).squeeze(-1)
    return result[nonzero_indices]


@torch.jit.script
def get_unpad_data(attention_mask: torch.Tensor):
    device = attention_mask.device
    seqlens_in_batch = get_max_seqlen_in_batch(attention_mask)
    indices = torch.nonzero(attention_mask.flatten()).flatten()
    max_seqlen_in_batch = seqlens_in_batch.max().item()
    cu_seqlens = (
        F.pad(torch.cumsum(seqlens_in_batch, dim=0, dtype=torch.int32), (1, 0))
        .to(device=device)
        .detach()
    )
    return (
        indices,
        cu_seqlens,
        max_seqlen_in_batch,
    )


def get_cu_seqlens(attn_mask):
    """generate a cumulative sequence length mask for flash attention using attn mask"""
    if len(attn_mask.shape) == 1:
        attn_mask = attn_mask.unsqueeze(0)

    device = attn_mask.device
    results = []
    max_seq_lens = []

    for row in attn_mask:
        # Exclude zeros to avoid adding their positions to the mask
        t_non_zeros = row[row != 0]
        # Find where the sequence number changes (including the first position)
        seq_change = torch.cat(
            [
                torch.tensor([1], dtype=torch.int32, device=device),
                t_non_zeros[1:] != t_non_zeros[:-1],
            ]
        )
        # Get the indices where the sequence changes
        change_indices = torch.cat(
            [
                (seq_change == 1).nonzero(as_tuple=True)[0],
                torch.tensor([len(t_non_zeros)], dtype=torch.int32, device=device),
            ]
        )
        # Calculate the sequence lengths
        seq_lengths = change_indices[1:] - change_indices[:-1]
        # Calculate the length of the final sequence or padding
        final_seq_length = len(row) - change_indices[-1]
        # Append the length of the final sequence or padding to seq_lengths
        if final_seq_length.item():
            seq_lengths = torch.cat(
                [
                    seq_lengths,
                    torch.tensor(
                        [final_seq_length.item()], dtype=torch.int32, device=device
                    ),
                ]
            )
        # Calculate the cumulative sequence lengths
        cu_seqlens = torch.cat(
            [torch.tensor([0], dtype=torch.int32, device=device), seq_lengths.cumsum(0)]
        )
        max_seq_len = (cu_seqlens[1:] - cu_seqlens[:-1]).max()
        results.append(cu_seqlens)
        max_seq_lens.append(max_seq_len)

    return torch.stack(results).to(dtype=torch.int32), torch.stack(max_seq_lens)


def get_cu_seqlens_from_pos_ids(position_ids):
    """generate a cumulative sequence length mask for flash attention using pos ids"""
    if len(position_ids.shape) == 1:
        position_ids = position_ids.unsqueeze(0)

    device = position_ids.device
    results = []
    max_seq_lens = []

    for row in position_ids:
        # Count the number of consecutive zeros from the right side
        padding_length = (row == 0).int().flip(dims=[0]).cumprod(dim=0).sum().item()

        # Adjust the row to exclude padding
        adjusted_row = row[:-padding_length] if padding_length else row.clone()

        # Find where the position resets to 0 (indicating a new sequence)
        seq_starts = torch.cat(
            [
                torch.tensor([True], dtype=torch.bool, device=device),
                adjusted_row[1:] == 0,
            ]
        )
        # Get the indices where the sequence starts
        start_indices = torch.cat(
            [
                torch.nonzero(seq_starts).unbind(dim=1)[0],
                torch.tensor([len(adjusted_row)], dtype=torch.int32, device=device),
            ]
        )
        # Calculate the sequence lengths
        seq_lengths = start_indices[1:] - start_indices[:-1]
        # Calculate the cumulative sequence lengths
        cu_seqlens = torch.cat(
            [torch.tensor([0], dtype=torch.int32, device=device), seq_lengths.cumsum(0)]
        )
        # Append the padding length to the cumulative sequence lengths
        if padding_length:
            cu_seqlens = torch.cat(
                [cu_seqlens, torch.tensor([len(row)], dtype=torch.int32, device=device)]
            )
        max_seq_len = (cu_seqlens[1:] - cu_seqlens[:-1]).max()
        results.append(cu_seqlens)
        max_seq_lens.append(max_seq_len)

    # Find the maximum value across all tensors
    max_value = max(t.max() for t in results)

    # Find the length of the longest tensor
    max_length = max(t.size(0) for t in results)

    # Pad each tensor to the same length and collect them in a list
    padded_results = [
        F.pad(t, (0, max_length - t.size(0)), "constant", max_value) for t in results
    ]

    return torch.stack(padded_results).to(dtype=torch.int32), torch.stack(max_seq_lens)


def set_module_name(model, name, value):
    if "." in name:
        parent_name = name.rsplit(".", 1)[0]
        child_name = name[len(parent_name) + 1 :]
        parent = model.get_submodule(parent_name)
    else:
        parent_name = ""
        parent = model
        child_name = name

    setattr(parent, child_name, value)


def mask_2d_to_4d(
    mask: torch.Tensor, dtype: torch.dtype, tgt_len: Optional[int] = None
):
    """
    Expands attention_mask from `[bsz, seq_len]` to `[bsz, 1, tgt_seq_len, src_seq_len]`.
    This expansion handles packed sequences so that sequences share the same attention mask integer value
    when they attend to each other within that sequence.
    This expansion transforms the mask to lower triangular form to prevent future peeking.
    """
    bsz, src_len = mask.size()
    tgt_len = tgt_len if tgt_len is not None else src_len

    mask = mask.unsqueeze(1).unsqueeze(2)
    mask = mask.expand(bsz, 1, tgt_len, src_len)

    # Create a binary mask from the original mask where zeros remain zeros and all other values are set to one
    binary_mask = torch.where(
        mask != 0,
        torch.tensor(1, device=mask.device).to(dtype),
        torch.tensor(0, device=mask.device).to(dtype),
    )

    # Create a block-diagonal mask.
    # we multiply by the binary mask so that 0's in the original mask are correctly excluded
    zero_one_mask = torch.eq(mask, mask.transpose(-1, -2)).int() * binary_mask

    # Now let's create a lower triangular mask of ones that will zero out the upper triangular part
    lower_triangular_ones = torch.tril(torch.ones((tgt_len, src_len), dtype=dtype)).to(
        mask.device
    )

    # Use the lower triangular mask to zero out the upper triangular part of the zero_one_mask
    masked_zero_one_mask = zero_one_mask * lower_triangular_ones

    return masked_zero_one_mask


def patched_prepare_4d_causal_attention_mask(
    attention_mask: Optional[torch.Tensor],
    *args,
):
    dtype = torch.bfloat16 if is_torch_bf16_gpu_available() else torch.float32
    return _prepare_4d_causal_attention_mask(
        mask_2d_to_4d(attention_mask, dtype=dtype),
        *args,
    )


def patched_prepare_4d_causal_attention_mask_for_sdpa(
    attention_mask: Optional[torch.Tensor],
    *args,
):
    dtype = torch.bfloat16 if is_torch_bf16_gpu_available() else torch.float32
    return _prepare_4d_causal_attention_mask_for_sdpa(
        mask_2d_to_4d(attention_mask, dtype=dtype),
        *args,
    )


In [8]:
def load_adapter(model, cfg, adapter, inference=False):
    # type: (PreTrainedModel, DictDefault, Optional[str], bool) -> Tuple[PreTrainedModel, Optional[PeftConfig]]

    if adapter is None:
        return model, None
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    if adapter in ["lora", "qlora"]:
        return load_lora(model, cfg, inference=inference)
    if adapter == "llama-adapter":
        return load_llama_adapter(model, cfg)

    raise NotImplementedError(f"{adapter} peft adapter not available")


def load_llama_adapter(model, cfg):
    # type: (PreTrainedModel, DictDefault) -> Tuple[PreTrainedModel, Optional[PeftConfig]]
    from peft import AdaptionPromptConfig, get_peft_model

    peft_config = AdaptionPromptConfig(
        adapter_layers=cfg.peft_adapter.layers,  # layers (L)
        adapter_len=cfg.peft_adapter.len,  # prompt length (K)
        task_type="CAUSAL_LM",
    )

    if cfg.lora_model_dir:
        model = PeftModel.from_pretrained(
            model,
            cfg.lora_model_dir,
            torch_dtype=torch.float16,
        )
    else:
        model = get_peft_model(model, peft_config)

    model.print_trainable_parameters()

    return model, peft_config


def find_all_linear_names(model):
    cls = (bnb.nn.Linear4bit, bnb.nn.Linear8bitLt, torch.nn.Linear, QuantLinear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if (
            isinstance(module, cls)
            or "Linear" in module.__class__.__name__
            and module.__class__.__name__ not in ("LlamaLinearScalingRotaryEmbedding",)
        ):
            # LOG.info(f"if문 isinstance(module, cls) or 'Linear' in module.__class__.__name__ and module.__class__.__name__ not in ('LlamaLinearScalingRotaryEmbedding',): {name}")
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    embedding_modules = get_linear_embedding_layers(model.config.model_type)
    output_embedding = embedding_modules[1]
    if output_embedding in lora_module_names:  # needed for 16-bit
        LOG.info(f"if output_embedding in lora_module_names:")
        lora_module_names.remove(output_embedding)

    return list(lora_module_names)


def load_lora(model, cfg, inference=False, config_only=False):
    # type: (PreTrainedModel, DictDefault, bool, bool) -> Tuple[Optional[PreTrainedModel], Optional[PeftConfig]]
    from peft import LoraConfig, get_peft_model

    lora_target_modules = list(cfg.lora_target_modules or [])

    if cfg.lora_target_linear:
        linear_names = find_all_linear_names(model)
        lora_target_modules = list(set(lora_target_modules + linear_names))

    lora_config_kwargs = {}
    # loftq_bits = cfg.peft and cfg.peft.loftq_config and cfg.peft.loftq_config.loftq_bits

    lora_config = LoraConfig(
        r=cfg.lora_r,
        lora_alpha=cfg.lora_alpha,
        target_modules=lora_target_modules,
        layers_to_transform=cfg.peft_layers_to_transform,
        lora_dropout=cfg.lora_dropout,
        fan_in_fan_out=cfg.lora_fan_in_fan_out,
        modules_to_save=cfg.lora_modules_to_save if cfg.lora_modules_to_save else None,
        bias="none",
        task_type="CAUSAL_LM",
        **lora_config_kwargs,
    )

    if cfg.lora_model_dir:
        model_kwargs: Any = {}
        # if cfg.lora_on_cpu:
        #     LOG.info(f"if cfg.lora_on_cpu:")
        #     model_kwargs["max_memory"] = {"cpu": "256GiB"}
        #     model_kwargs["device_map"] = {"": "cpu"}
        model = PeftModel.from_pretrained(
            model,
            cfg.lora_model_dir,
            is_trainable=(not inference),
            **model_kwargs,
        )
    else:
        model = get_peft_model(model, lora_config)

    model.print_trainable_parameters()

    return model, lora_config

In [9]:
"""Flash attention monkey patch for llama model"""

# copied from https://github.com/lm-sys/FastChat/blob/main/fastchat/train/llama_flash_attn_monkey_patch.py

import logging
import warnings
from functools import partial
from typing import List, Optional, Tuple, Union

import torch
import torch.nn.functional as F
import transformers
from einops import rearrange
from flash_attn.bert_padding import pad_input, unpad_input
from transformers.modeling_outputs import BaseModelOutputWithPast
from transformers.models.llama.modeling_llama import LlamaAttention
from transformers.models.llama.modeling_llama import (
    LlamaDecoderLayer as OriginalLlamaDecoderLayer,
)
from transformers.models.llama.modeling_llama import (
    LlamaMLP,
    apply_rotary_pos_emb,
    repeat_kv,
)
from xformers.ops import SwiGLU

try:
    from flash_attn.flash_attn_interface import (  # pylint: disable=ungrouped-imports
        flash_attn_kvpacked_func,
        flash_attn_varlen_kvpacked_func,
        flash_attn_varlen_qkvpacked_func,
    )
except ImportError:
    from flash_attn.flash_attn_interface import (
        flash_attn_unpadded_kvpacked_func as flash_attn_varlen_kvpacked_func,
    )
    from flash_attn.flash_attn_interface import (
        flash_attn_unpadded_qkvpacked_func as flash_attn_varlen_qkvpacked_func,
    )


LOG = logging.getLogger("axolotl")


def replace_llama_mlp_with_swiglu(model):
    for name, module in model.named_modules():
        if isinstance(module, LlamaMLP):
            mlp = FusedMLP(
                module.config, module.gate_proj, module.up_proj, module.down_proj
            )
            set_module_name(model, name, mlp)


def replace_llama_qkv_with_fused(model):
    for name, module in model.named_modules():
        if isinstance(module, LlamaAttention):
            qkv = FusedAttention(
                module.config,
                module.q_proj,
                module.k_proj,
                module.v_proj,
                module.o_proj,
            )
            set_module_name(model, name, qkv)


def replace_llama_attn_with_flash_attn(
    packed: Optional[bool] = False,
    cross_entropy: Optional[bool] = False,
    rms_norm: Optional[bool] = False,
    use_shifted_sparse_attn: Optional[bool] = False,
):
    transformers.models.llama.modeling_llama.LlamaModel._prepare_decoder_attention_mask = (  # pylint: disable=protected-access
        _prepare_decoder_attention_mask
    )
    if use_shifted_sparse_attn:
        transformers.models.llama.modeling_llama.LlamaAttention.forward = (
            flashattn_forward_with_s2attn
        )
    else:
        transformers.models.llama.modeling_llama.LlamaAttention.forward = (
            flashattn_forward
        )

    if packed:
        transformers.models.llama.modeling_llama.LlamaDecoderLayer = LlamaDecoderLayer
        transformers.models.llama.modeling_llama.LlamaModel.forward = (
            llama_model_forward
        )

    # skip only if explicitly disabled
    if cross_entropy:
        try:
            from flash_attn.losses.cross_entropy import CrossEntropyLoss

            LOG.info("patching with flash_attn.losses.cross_entropy")
            transformers.models.llama.modeling_llama.CrossEntropyLoss = partial(
                CrossEntropyLoss, inplace_backward=True
            )
        except ImportError:
            LOG.info(
                "optimized flash-attention CrossEntropyLoss not found (run `pip install 'git+https://github.com/Dao-AILab/flash-attention.git#egg=xentropy_cuda_lib&subdirectory=csrc/xentropy'`)"
            )

    # skip only if explicitly disabled
    if rms_norm:
        try:
            from flash_attn.ops.rms_norm import RMSNorm

            class LlamaRMSNorm(RMSNorm):
                """Patched LLamaRMSNorm"""

                def __init__(self, hidden_size, eps=1e-6):
                    super().__init__(hidden_size, eps=eps)

            LOG.info("patching with flash_attn.ops.rms_norm")
            transformers.models.llama.modeling_llama.LlamaRMSNorm = LlamaRMSNorm
        except ImportError:
            LOG.info(
                "optimized flash-attention RMSNorm not found (run `pip install 'git+https://github.com/Dao-AILab/flash-attention.git#egg=dropout_layer_norm&subdirectory=csrc/layer_norm'`)"
            )


class FusedAttention(LlamaAttention):
    """
    Fused QKV Attention layer for incrementally improved training efficiency
    """

    def __init__(
        self,
        config,
        q: torch.nn.Linear,  # pylint: disable=invalid-name
        k: torch.nn.Linear,  # pylint: disable=invalid-name
        v: torch.nn.Linear,  # pylint: disable=invalid-name
        o: torch.nn.Linear,  # pylint: disable=invalid-name
    ):
        super().__init__(config)
        self.config = config
        self.init_device = next(iter(q.state_dict().values())).device

        # define equivalent fused qkv projection
        self.out_features: List[int] = [q.out_features, k.out_features, v.out_features]
        self.qkv_proj = torch.nn.Linear(
            q.in_features, sum(self.out_features), device=self.init_device, bias=False
        )
        self.o_proj = o

        # overwrite initialized weights with pretrained weights
        self.qkv_proj.weight.data = torch.cat(
            (q.weight.data, k.weight.data, v.weight.data), dim=0
        )

    def _post_training(self, model, name):
        q_proj, k_proj, v_proj = torch.split(
            self.qkv_proj.weight.data, self.out_features, dim=0
        )

        new_attn = LlamaAttention(self.config)
        new_attn.q_proj.weight.data = q_proj
        new_attn.k_proj.weight.data = k_proj
        new_attn.v_proj.weight.data = v_proj
        new_attn.o_proj.weight.data = self.o_proj.weight.data

        set_module_name(model, name, new_attn)


class FusedMLP(torch.nn.Module):
    """
    Fused MLP layer for incrementally improved training efficiency
    """

    def __init__(
        self,
        config,
        gate_proj: torch.nn.Linear,
        up_proj: torch.nn.Linear,
        down_proj: torch.nn.Linear,
    ):
        super().__init__()
        self.config = config
        self.swiglu = SwiGLU(
            in_features=config.hidden_size,
            hidden_features=config.intermediate_size,
            bias=False,
            _pack_weights=True,
        )
        # overwrite initialized weights with pretrained weights
        self.swiglu.w12.weight.data = torch.cat(
            (gate_proj.weight.data, up_proj.weight.data), dim=0
        )
        self.swiglu.w3.weight.data = down_proj.weight.data

    def _post_training(self, model, name):
        w1, w2 = torch.split(  # pylint: disable=invalid-name
            self.swiglu.w12.weight.data, self.config.intermediate_size, dim=0
        )

        # Assign the split weights back to the original layers
        new_mlp = LlamaMLP(self.config)
        new_mlp.gate_proj.weight.data = w1
        new_mlp.up_proj.weight.data = w2
        new_mlp.down_proj.weight.data = self.swiglu.w3.weight.data

        set_module_name(model, name, new_mlp)

    def forward(self, x: torch.Tensor) -> torch.Tensor:  # pylint: disable=invalid-name
        return self.swiglu(x)


# Disable the transformation of the attention mask in LlamaModel as the flash attention
# requires the attention mask to be the same as the key_padding_mask
def _prepare_decoder_attention_mask(
    self,
    attention_mask,
    input_shape,
    inputs_embeds,
    past_key_values_length,
):  # pylint: disable=unused-argument
    # [bsz, seq_len]
    return attention_mask


GROUP_SIZE_RATIO = 1 / 4


def flashattn_forward_with_s2attn(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    past_key_value: Optional[Tuple[torch.Tensor]] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
    padding_mask: Optional[torch.LongTensor] = None,  # pylint: disable=unused-argument
    cu_seqlens: Optional[torch.Tensor] = None,  # pylint: disable=unused-argument
    max_seqlen: Optional[torch.Tensor] = None,  # pylint: disable=unused-argument
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    """Input shape: Batch x Time x Channel

    From: https://github.com/dvlab-research/LongLoRA/blob/main/llama_attn_replace.py

    attention_mask: [bsz, q_len]

    `cu_seqlens` will be ignored if provided
    `max_seqlen` will be ignored if provided
    """
    if output_attentions:
        warnings.warn(
            "Output attentions is not supported for patched `LlamaAttention`, returning `None` instead."
        )

    bsz, q_len, _ = hidden_states.size()

    query_states = (
        self.q_proj(hidden_states)
        .view(bsz, q_len, self.num_heads, self.head_dim)
        .transpose(1, 2)
    )
    key_states = (
        self.k_proj(hidden_states)
        .view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        .transpose(1, 2)
    )
    value_states = (
        self.v_proj(hidden_states)
        .view(bsz, q_len, self.num_key_value_heads, self.head_dim)
        .transpose(1, 2)
    )
    # [bsz, q_len, nh, hd]
    # [bsz, nh, q_len, hd]
    # pylint: disable=duplicate-code

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value[0].shape[-2]
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
    query_states, key_states = apply_rotary_pos_emb(
        query_states, key_states, cos, sin, position_ids
    )

    # Past Key value support
    if past_key_value is not None:
        # reuse k, v, self_attention
        key_states = torch.cat([past_key_value[0], key_states], dim=2)
        value_states = torch.cat([past_key_value[1], value_states], dim=2)

    past_key_value = (key_states, value_states) if use_cache else None

    # repeat k/v heads if n_kv_heads < n_heads
    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    # Flash attention codes from
    # https://github.com/HazyResearch/flash-attention/blob/main/flash_attn/flash_attention.py

    # transform the data into the format required by flash attention
    qkv = torch.stack(
        [query_states, key_states, value_states], dim=2
    )  # [bsz, nh, 3, q_len, hd]
    qkv = qkv.transpose(1, 3)  # [bsz, q_len, 3, nh, hd]

    # We have disabled _prepare_decoder_attention_mask in LlamaModel
    # the attention_mask should be the same as the key_padding_mask

    key_padding_mask = attention_mask.repeat(2, 1)
    nheads = qkv.shape[-2]
    # shift

    group_size = int(q_len * GROUP_SIZE_RATIO)
    if q_len % group_size > 0:
        raise ValueError(
            f"q_len {q_len} should be divisible by group size {group_size}."
        )

    qkv = (
        qkv.reshape(bsz, q_len, 3, 2, self.num_heads // 2, self.head_dim)
        .permute(0, 3, 1, 2, 4, 5)
        .reshape(bsz * 2, q_len, 3, self.num_heads // 2, self.head_dim)
    )
    x = rearrange(  # pylint: disable=invalid-name
        qkv, "b s three h d -> b s (three h d)"
    )
    x_unpad, indices, cu_q_lens, max_s = unpad_input(x, key_padding_mask)
    cu_q_len_tmp = torch.arange(
        0, max_s, group_size, device=key_padding_mask.device, dtype=cu_q_lens.dtype
    )
    cu_q_len_tmp = torch.stack([cu_q_len_tmp, cu_q_len_tmp + group_size // 2]).repeat(
        bsz, 1
    ) + cu_q_lens[:-1].unsqueeze(-1)
    cu_q_lens = torch.cat([cu_q_len_tmp, cu_q_lens[1:].unsqueeze(-1)], dim=-1).view(-1)

    x_unpad = rearrange(
        x_unpad, "nnz (three h d) -> nnz three h d", three=3, h=nheads // 2
    )
    output_unpad = flash_attn_varlen_qkvpacked_func(
        x_unpad, cu_q_lens, group_size, 0.0, softmax_scale=None, causal=True
    )
    output = rearrange(
        pad_input(
            rearrange(output_unpad, "nnz h d -> nnz (h d)"), indices, bsz * 2, q_len
        ),
        "b s (h d) -> b s h d",
        h=nheads // 2,
    )
    output = (
        output.reshape(bsz, 2, q_len, nheads // 2, self.head_dim)
        .transpose(1, 2)
        .reshape(bsz, q_len, nheads, self.head_dim)
    )
    return self.o_proj(rearrange(output, "b s h d -> b s (h d)")), None, past_key_value


def flashattn_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    past_key_value: Optional[Tuple[torch.Tensor]] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
    padding_mask: Optional[torch.LongTensor] = None,  # pylint: disable=unused-argument
    cu_seqlens: Optional[torch.Tensor] = None,
    max_seqlen: Optional[torch.Tensor] = None,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    """Input shape: Batch x Time x Channel

    attention_mask: [bsz, q_len]
    """
    # pylint: disable=duplicate-code
    bsz, q_len, _ = hidden_states.size()

    if not hasattr(self, "pretraining_tp"):
        self.pretraining_tp = 1

    if self.pretraining_tp > 1:
        key_value_slicing = (
            self.num_key_value_heads * self.head_dim
        ) // self.pretraining_tp
        query_slices = self.q_proj.weight.split(
            (self.num_heads * self.head_dim) // self.pretraining_tp, dim=0
        )
        key_slices = self.k_proj.weight.split(key_value_slicing, dim=0)
        value_slices = self.v_proj.weight.split(key_value_slicing, dim=0)

        query_states = [
            F.linear(hidden_states, query_slices[i]) for i in range(self.pretraining_tp)
        ]
        query_states = torch.cat(query_states, dim=-1)

        key_states = [
            F.linear(hidden_states, key_slices[i]) for i in range(self.pretraining_tp)
        ]
        key_states = torch.cat(key_states, dim=-1)

        value_states = [
            F.linear(hidden_states, value_slices[i]) for i in range(self.pretraining_tp)
        ]
        value_states = torch.cat(value_states, dim=-1)

    else:
        if isinstance(self, FusedAttention):
            query_states, key_states, value_states = self.qkv_proj(hidden_states).split(
                self.out_features, dim=-1
            )
        else:
            query_states = self.q_proj(hidden_states)
            key_states = self.k_proj(hidden_states)
            value_states = self.v_proj(hidden_states)

    query_states = query_states.view(
        bsz, q_len, self.num_heads, self.head_dim
    ).transpose(1, 2)
    key_states = key_states.view(
        bsz, q_len, self.num_key_value_heads, self.head_dim
    ).transpose(1, 2)
    value_states = value_states.view(
        bsz, q_len, self.num_key_value_heads, self.head_dim
    ).transpose(1, 2)
    # [bsz, q_len, nh, hd]
    # [bsz, nh, q_len, hd]

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value[0].shape[-2]

    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
    query_states, key_states = apply_rotary_pos_emb(
        query_states, key_states, cos, sin, position_ids
    )
    # [bsz, nh, t, hd]

    if past_key_value is not None:
        # reuse k, v, self_attention
        key_states = torch.cat([past_key_value[0], key_states], dim=2)
        value_states = torch.cat([past_key_value[1], value_states], dim=2)

    past_key_value = (key_states, value_states) if use_cache else None

    # repeat k/v heads if n_kv_heads < n_heads
    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    if output_attentions:
        warnings.warn(
            "Output attentions is not supported for patched `LlamaAttention`, returning `None` instead."
        )

    #
    # flash-attn v2 start
    #

    if self.training:
        # during training q,k,v always have same seqlen
        assert key_states.shape == query_states.shape
        is_causal = True
    else:
        # turn off FA causal mask after first inference autoregressive iteration
        # only on first autoregressive step q,k,v have same seqlen
        is_causal = key_states.shape == query_states.shape

    dropout_rate = 0.0 if not self.training else getattr(self, "attention_dropout", 0.0)

    if cu_seqlens is not None and max_seqlen is not None and cu_seqlens.dim() == 1:
        # special handling using sample packing
        qkv = torch.stack(
            [query_states, key_states, value_states], dim=2
        )  # [bsz, nh, 3, q_len, hd]
        qkv = qkv.transpose(1, 3)  # [bsz, q_len, 3, nh, hd]
        qkv = rearrange(qkv, "b s ... -> (b s) ...")

        output = flash_attn_varlen_qkvpacked_func(
            qkv,
            cu_seqlens,
            max_seqlen,
            dropout_p=dropout_rate,
            softmax_scale=None,
            causal=True,
        )
        output = rearrange(output, "(b s) ... -> b s ...", b=bsz)
    elif query_states.shape == key_states.shape:
        query_states = query_states.transpose(1, 2)
        key_states = key_states.transpose(1, 2)
        value_states = value_states.transpose(1, 2)
        qkv_unpad, cu_seqlens_q, max_seqlen_q, _, output_pad_fn = generate_qkv(
            query_states,
            key_states,
            value_states,
            qkvpacked=True,
            # We have disabled _prepare_decoder_attention_mask in LlamaModel
            # the attention_mask should be the same as the key_padding_mask
            key_padding_mask=attention_mask,
            query_padding_mask=attention_mask[:, -query_states.size(1) :]
            if attention_mask is not None
            else None,
        )
        output_unpad = flash_attn_varlen_qkvpacked_func(
            qkv_unpad,
            cu_seqlens_q,
            max_seqlen_q,
            dropout_p=dropout_rate,
            softmax_scale=None,
            causal=is_causal,
        )
        output = output_pad_fn(output_unpad)
    else:
        query_states = query_states.transpose(1, 2)
        key_states = key_states.transpose(1, 2)
        value_states = value_states.transpose(1, 2)
        if attention_mask is None or attention_mask.all().item():
            output = flash_attn_kvpacked_func(
                query_states,
                torch.stack([key_states, value_states], 2),
                dropout_p=dropout_rate,
                causal=is_causal,
            )
        else:
            (  # pylint: disable=unbalanced-tuple-unpacking
                q_unpad,
                kv_unpad,
                cu_seqlens_q,
                cu_seqlens_k,
                max_seqlen_q,
                max_seqlen_k,
                _,
                _,
                output_pad_fn,
            ) = generate_qkv(
                query_states,
                key_states,
                value_states,
                kvpacked=True,
                key_padding_mask=attention_mask,
                query_padding_mask=attention_mask[:, -query_states.size(1) :]
                if attention_mask is not None
                else None,
            )
            if q_unpad.dtype != kv_unpad.dtype:
                kv_unpad = kv_unpad.to(q_unpad.dtype)
            output_unpad = flash_attn_varlen_kvpacked_func(
                q_unpad,
                kv_unpad,
                cu_seqlens_q,
                cu_seqlens_k,
                max_seqlen_q,
                max_seqlen_k,
                dropout_p=dropout_rate,
                softmax_scale=None,
                causal=is_causal,
            )
            output = output_pad_fn(output_unpad)

    attn_output = output
    if attn_output.size() != (bsz, q_len, self.num_heads, self.head_dim):
        raise ValueError(
            f"`attn_output` should be of size {(bsz, q_len, self.num_heads, self.head_dim)}, but is"
            f" {attn_output.size()}"
        )
    attn_output = rearrange(attn_output, "b s h d -> b s (h d)")

    #
    # flash-attn v2 end
    #

    if self.pretraining_tp > 1:
        attn_output = attn_output.split(self.hidden_size // self.pretraining_tp, dim=2)
        o_proj_slices = self.o_proj.weight.split(
            self.hidden_size // self.pretraining_tp, dim=1
        )
        attn_output = sum(
            F.linear(attn_output[i], o_proj_slices[i])
            for i in range(self.pretraining_tp)
        )
    else:
        attn_output = self.o_proj(attn_output)

    return attn_output, None, past_key_value


# based on https://github.com/Dao-AILab/flash-attention/blob/364a5b/tests/test_flash_attn.py#L38
def generate_qkv(
    q,
    k,
    v,
    query_padding_mask=None,
    key_padding_mask=None,
    kvpacked=False,
    qkvpacked=False,
):  # pylint: disable=invalid-name,unnecessary-lambda-assignment
    """
    Arguments:
        q: (batch_size, seqlen_q, nheads, d)
        k: (batch_size, seqlen_k, nheads_k, d)
        v: (batch_size, seqlen_k, nheads_k, d)
        query_padding_mask: (batch_size, seqlen), bool
        key_padding_mask: (batch_size, seqlen), bool
    """
    assert not (kvpacked and qkvpacked)
    batch_size, seqlen_q, nheads, d = q.shape
    _, seqlen_k, nheads_k, _ = k.shape
    assert k.shape == (batch_size, seqlen_k, nheads_k, d)
    assert v.shape == (batch_size, seqlen_k, nheads_k, d)

    if query_padding_mask is not None:
        q_unpad, indices_q, cu_seqlens_q, max_seqlen_q = unpad_input(
            q, query_padding_mask
        )

        output_pad_fn = lambda output_unpad: pad_input(  # noqa: E731
            output_unpad, indices_q, batch_size, seqlen_q
        )

    else:
        q_unpad = rearrange(q, "b s h d -> (b s) h d")
        cu_seqlens_q = torch.arange(
            0,
            (batch_size + 1) * seqlen_q,
            step=seqlen_q,
            dtype=torch.int32,
            device=q_unpad.device,
        )
        max_seqlen_q = seqlen_q

        output_pad_fn = lambda output_unpad: rearrange(  # noqa: E731
            output_unpad, "(b s) h d -> b s h d", b=batch_size
        )

    if key_padding_mask is not None:
        k_unpad, _, cu_seqlens_k, max_seqlen_k = unpad_input(k, key_padding_mask)
        v_unpad, _, _, _ = unpad_input(v, key_padding_mask)
    else:
        k_unpad = rearrange(k, "b s h d -> (b s) h d")
        v_unpad = rearrange(v, "b s h d -> (b s) h d")
        cu_seqlens_k = torch.arange(
            0,
            (batch_size + 1) * seqlen_k,
            step=seqlen_k,
            dtype=torch.int32,
            device=k_unpad.device,
        )
        max_seqlen_k = seqlen_k

    if qkvpacked:
        assert nheads == nheads_k
        qkv_unpad = torch.stack([q_unpad, k_unpad, v_unpad], dim=1)
        qkv = torch.stack([q, k, v], dim=2)
        return (qkv_unpad, cu_seqlens_q, max_seqlen_q, qkv, output_pad_fn)

    if kvpacked:
        kv_unpad = torch.stack([k_unpad, v_unpad], dim=1)
        kv = torch.stack([k, v], dim=2)
        return (
            q_unpad,
            kv_unpad,
            cu_seqlens_q,
            cu_seqlens_k,
            max_seqlen_q,
            max_seqlen_k,
            q,
            kv,
            output_pad_fn,
        )

    return (
        q_unpad,
        k_unpad,
        v_unpad,
        cu_seqlens_q,
        cu_seqlens_k,
        max_seqlen_q,
        max_seqlen_k,
        q,
        k,
        v,
        output_pad_fn,
    )


def llama_model_forward(
    self,
    input_ids: torch.LongTensor = None,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, BaseModelOutputWithPast]:
    output_attentions = (
        output_attentions
        if output_attentions is not None
        else self.config.output_attentions
    )
    output_hidden_states = (
        output_hidden_states
        if output_hidden_states is not None
        else self.config.output_hidden_states
    )
    use_cache = use_cache if use_cache is not None else self.config.use_cache

    return_dict = (
        return_dict if return_dict is not None else self.config.use_return_dict
    )

    # retrieve input_ids and inputs_embeds
    if input_ids is not None and inputs_embeds is not None:
        raise ValueError(
            "You cannot specify both decoder_input_ids and decoder_inputs_embeds at the same time"
        )
    if input_ids is not None:
        batch_size, seq_length = input_ids.shape
    elif inputs_embeds is not None:
        batch_size, seq_length, _ = inputs_embeds.shape
    else:
        raise ValueError(
            "You have to specify either decoder_input_ids or decoder_inputs_embeds"
        )

    seq_length_with_past = seq_length
    past_key_values_length = 0

    if past_key_values is not None:
        past_key_values_length = past_key_values[0][0].shape[2]
        seq_length_with_past = seq_length_with_past + past_key_values_length

    cu_seqlens = None
    max_seqlen = None
    if position_ids is None:
        device = input_ids.device if input_ids is not None else inputs_embeds.device
        position_ids = torch.arange(
            past_key_values_length,
            seq_length + past_key_values_length,
            dtype=torch.long,
            device=device,
        )
        position_ids = position_ids.unsqueeze(0).view(-1, seq_length)
    else:
        position_ids = position_ids.view(-1, seq_length).long()
        cu_seqlens, max_seqlen = get_cu_seqlens_from_pos_ids(position_ids)
        cu_seqlens = cu_seqlens.squeeze()

    if inputs_embeds is None:
        inputs_embeds = self.embed_tokens(input_ids)
    # embed positions
    if attention_mask is None:
        attention_mask = torch.ones(
            (batch_size, seq_length_with_past),
            dtype=torch.bool,
            device=inputs_embeds.device,
        )
        padding_mask = None
    else:
        if 0 in attention_mask:
            padding_mask = attention_mask
        else:
            padding_mask = None

    attention_mask = (
        self._prepare_decoder_attention_mask(  # pylint: disable=protected-access
            attention_mask,
            (batch_size, seq_length),
            inputs_embeds,
            past_key_values_length,
        )
    )

    hidden_states = inputs_embeds

    if self.gradient_checkpointing and self.training:
        if use_cache:
            transformers.logger.warning_once(
                "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
            )
            use_cache = False

    # decoder layers
    all_hidden_states = () if output_hidden_states else None
    all_self_attns = () if output_attentions else None
    next_decoder_cache = () if use_cache else None

    for idx, decoder_layer in enumerate(self.layers):
        if output_hidden_states:
            all_hidden_states += (hidden_states,)

        past_key_value = past_key_values[idx] if past_key_values is not None else None

        if self.gradient_checkpointing and self.training:

            def create_custom_forward(module):
                def custom_forward(*inputs):
                    # None for past_key_value
                    return module(
                        *inputs,
                    )

                return custom_forward

            layer_outputs = torch.utils.checkpoint.checkpoint(
                create_custom_forward(decoder_layer),
                hidden_states,
                attention_mask,
                position_ids,
                past_key_value,
                output_attentions,
                None,
                padding_mask,
                cu_seqlens,
                max_seqlen,
            )
        else:
            layer_outputs = decoder_layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_value=past_key_value,
                output_attentions=output_attentions,
                use_cache=use_cache,
                padding_mask=padding_mask,
                cu_seqlens=cu_seqlens,
                max_seqlen=max_seqlen,
            )

        hidden_states = layer_outputs[0]

        if use_cache:
            next_decoder_cache += (layer_outputs[2 if output_attentions else 1],)

        if output_attentions:
            all_self_attns += (layer_outputs[1],)

    hidden_states = self.norm(hidden_states)

    # add hidden states from the last decoder layer
    if output_hidden_states:
        all_hidden_states += (hidden_states,)

    next_cache = next_decoder_cache if use_cache else None
    if not return_dict:
        return tuple(
            v
            for v in [hidden_states, next_cache, all_hidden_states, all_self_attns]
            if v is not None
        )
    return BaseModelOutputWithPast(
        last_hidden_state=hidden_states,
        past_key_values=next_cache,
        hidden_states=all_hidden_states,
        attentions=all_self_attns,
    )


class LlamaDecoderLayer(OriginalLlamaDecoderLayer):
    """
    patched version of LlamaDecoderLayer to pass through the precalculated cu_seqlens
    """

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: Optional[bool] = False,
        use_cache: Optional[bool] = False,
        padding_mask: Optional[torch.LongTensor] = None,
        cu_seqlens: Optional[torch.Tensor] = None,
        max_seqlen: Optional[torch.Tensor] = None,
    ) -> Tuple[
        torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]
    ]:
        """
        Args:
            hidden_states (`torch.FloatTensor`): input to the layer of shape `(batch, seq_len, embed_dim)`
            attention_mask (`torch.FloatTensor`, *optional*): attention mask of size
                `(batch, 1, tgt_len, src_len)` where padding elements are indicated by very large negative values.
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
            use_cache (`bool`, *optional*):
                If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                (see `past_key_values`).
            past_key_value (`Tuple(torch.FloatTensor)`, *optional*): cached past key and value projection states
            cu_seqlens (`torch.Tensor`, *optional*) cumulative sequence len when packing
        """

        residual = hidden_states

        hidden_states = self.input_layernorm(hidden_states)

        # Self Attention
        hidden_states, self_attn_weights, present_key_value = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            use_cache=use_cache,
            padding_mask=padding_mask,
            cu_seqlens=cu_seqlens,
            max_seqlen=max_seqlen,
        )
        hidden_states = residual + hidden_states

        # Fully Connected
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states

        outputs = (hidden_states,)

        if output_attentions:
            outputs += (self_attn_weights,)

        if use_cache:
            outputs += (present_key_value,)

        return outputs


In [10]:
"""Benchmarking and measurement utilities"""
import functools

import pynvml
import torch
from pynvml.nvml import NVMLError


def check_cuda_device(default_value):
    """
    wraps a function and returns the default value instead of running the
    wrapped function if cuda isn't available or the device is auto
    :param default_value:
    :return:
    """

    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            device = kwargs.get("device", args[0] if args else None)

            if (
                device is None
                or not torch.cuda.is_available()
                or device == "auto"
                or torch.device(device).type == "cpu"
            ):
                return default_value

            return func(*args, **kwargs)

        return wrapper

    return deco


@check_cuda_device(0.0)
def gpu_memory_usage(device=0):
    return torch.cuda.memory_allocated(device) / 1024.0**3


@check_cuda_device((0.0, 0.0, 0.0))
def gpu_memory_usage_all(device=0):
    usage = torch.cuda.memory_allocated(device) / 1024.0**3
    reserved = torch.cuda.memory_reserved(device) / 1024.0**3
    smi = gpu_memory_usage_smi(device)
    return usage, reserved - usage, max(0, smi - reserved)


def mps_memory_usage_all():
    usage = torch.mps.current_allocated_memory() / 1024.0**3
    reserved = torch.mps.driver_allocated_memory() / 1024.0**3
    return usage, reserved - usage, 0


@check_cuda_device(0.0)
def gpu_memory_usage_smi(device=0):
    if isinstance(device, torch.device):
        device = device.index
    if isinstance(device, str) and device.startswith("cuda:"):
        device = int(device[5:])
    try:
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(device)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        return info.used / 1024.0**3
    except NVMLError:
        return 0.0


def log_gpu_memory_usage(log, msg, device):
    if torch.backends.mps.is_available():
        usage, cache, misc = mps_memory_usage_all()
    else:
        usage, cache, misc = gpu_memory_usage_all(device)
    extras = []
    if cache > 0:
        extras.append(f"+{cache:.03f}GB cache")
    if misc > 0:
        extras.append(f"+{misc:.03f}GB misc")
    log.info(
        f"GPU memory usage {msg}: {usage:.03f}GB ({', '.join(extras)})", stacklevel=2
    )
    return usage, cache, misc

In [14]:
def load_model(
    cfg,
    tokenizer: PreTrainedTokenizerBase,
    inference: bool = False,
    reference_model: bool = False,
) -> Tuple[PreTrainedModel, Optional[PeftConfig]]:
    """
    Load a model for a given configuration and tokenizer.
    """
    base_model = cfg.base_model
    model_type = cfg.model_type
    model_config = load_model_config(cfg)

    # TODO refactor as a kwarg
    load_in_8bit = cfg.load_in_8bit


    if cfg.sample_packing and cfg.s2_attention:
        raise ValueError(
            "Received `sample_packing=true` and `s2_attention=true`; however, \
        shifted-sparse attention does not currently support sample packing."
        )

    model_kwargs: Dict[str, Any] = {}

    # if cfg.model_kwargs:
    #     for key, val in cfg.model_kwargs.items():
    #         model_kwargs[key] = val

    max_memory = cfg.max_memory
    device_map = cfg.device_map
    if device_map:
        LOG.info(f"Device map specified: {device_map}")
    else:
        LOG.info("No device map specified, using default GPU allocation.")
    LOG.info("device_map: %s", device_map)
    
    if cfg.gpu_memory_limit:
        LOG.info(f"GPU memory limit specified: {cfg.gpu_memory_limit}")
        gpu_memory_limit = (
            str(cfg.gpu_memory_limit) + "GiB"
            if isinstance(cfg.gpu_memory_limit, int)
            else cfg.gpu_memory_limit
        )

        max_memory = {}
        for i in range(torch.cuda.device_count()):
            max_memory[i] = gpu_memory_limit

        LOG.info(f"max_memory[i]: {max_memory}")
        max_memory["cpu"] = "256GiB"  # something sufficiently large to fit anything

    if max_memory is not None:
        # Based on https://github.com/togethercomputer/OpenChatKit/blob/main/inference/bot.py
        LOG.info("if문 max_memory is not None")
        from accelerate import infer_auto_device_map, init_empty_weights

        with init_empty_weights():
            model_canvas = AutoModelForCausalLM.from_config(model_config)
        model_canvas.tie_weights()
        device_map = infer_auto_device_map(
            model_canvas,
            max_memory=max_memory,
            dtype=cfg.torch_dtype,
        )
        # We can discard max_memory now as we have a device map set up for us
        max_memory = None

    model_kwargs["device_map"] = device_map
    model_kwargs["torch_dtype"] = cfg.torch_dtype

    # if torch.backends.mps.is_available():
    #     model_kwargs["device_map"] = "mps:0"

    if is_deepspeed_zero3_enabled():
        LOG.info(f"if is_deepspeed_zero3_enabled():")
        del model_kwargs["device_map"]


    if cfg.adapter == "qlora" and cfg.load_in_4bit:
        bnb_config = {
            "load_in_4bit": True,
            "llm_int8_threshold": 6.0,
            "llm_int8_has_fp16_weight": False,
            "bnb_4bit_compute_dtype": cfg.torch_dtype,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
        }

        if cfg.bnb_config_kwargs:
            bnb_config.update(cfg.bnb_config_kwargs)

        model_kwargs["quantization_config"] = BitsAndBytesConfig(
            **bnb_config,
        )


    if cfg.load_in_8bit and cfg.adapter is not None:
        model_kwargs["load_in_8bit"] = True
    if cfg.load_in_4bit and cfg.adapter is not None:
        model_kwargs["load_in_4bit"] = True

    if "quantization_config" in model_kwargs or cfg.gptq:
        LOG.info("if 'quantization_config' in model_kwargs or cfg.gptq:")
        if "load_in_8bit" in model_kwargs:
            del model_kwargs["load_in_8bit"]
        if "load_in_4bit" in model_kwargs:
            del model_kwargs["load_in_4bit"]
            
    
    # sample packing uses custom FA2 patch
    if cfg.flash_attention:
        if not cfg.sample_packing:
            if cfg.s2_attention:
                pass
            # most other models support flash attention, we can define exceptions as they come up
            model_kwargs["attn_implementation"] = "flash_attention_2"
            model_config._attn_implementation = (  # pylint: disable=protected-access
                "flash_attention_2"
            )
        else:
            if model_config.model_type in ["mixtral", "qwen2", "falcon", "phi"]:
                LOG.info(f"if model_config.model_type in SUPPORTED_MULTIPACK_MODEL_TYPES:")
                model_kwargs["attn_implementation"] = "flash_attention_2"
                model_config._attn_implementation = (  # pylint: disable=protected-access
                    "flash_attention_2"
                )
            else:
                LOG.info(f"if not model_config.model_type in SUPPORTED_MULTIPACK_MODEL_TYPES:")
                model_kwargs["attn_implementation"] = "eager"
                model_config._attn_implementation = (  # pylint: disable=protected-access
                    "eager"
                )
    elif cfg.sdp_attention:
        LOG.info(f"if cfg.sdp_attention:")
        model_kwargs["attn_implementation"] = "sdpa"
        model_config._attn_implementation = "sdpa"  # pylint: disable=protected-access
    elif cfg.eager_attention:
        LOG.info(f"if cfg.eager_attention:")
        model_kwargs["attn_implementation"] = "eager"
        model_config._attn_implementation = "eager"  # pylint: disable=protected-access

    try:
        if (
            model_config.model_type == "llama"
            and not cfg.trust_remote_code
        ):
            LOG.info("Loading Llama model with specific settings")
            from transformers import LlamaForCausalLM

            model = LlamaForCausalLM.from_pretrained(
                base_model,
                config=model_config,
                **model_kwargs,
            )

        elif model_type and not cfg.trust_remote_code:
            model = getattr(transformers, model_type).from_pretrained(
                base_model,
                config=model_config,
                trust_remote_code=cfg.trust_remote_code or False,
                **model_kwargs,
            )
        else:
            # Shouldn't be a problem most of the time. will obviously error if the model doesn't support this
            # when training starts
            if (
                hasattr(model_config, "max_seq_len")
                and model_config.max_seq_len
                and cfg.sequence_len > model_config.max_seq_len
            ):
                LOG.info(f"if문 hasattr(model_config, "'max_seq_len'") and model_config.max_seq_len and cfg.sequence_len > model_config.max_seq_len:")
                model_config.max_seq_len = cfg.sequence_len
                LOG.warning(f"increasing context length to {cfg.sequence_len}")
            elif (
                hasattr(model_config, "max_sequence_length")
                and model_config.max_sequence_length
                and cfg.sequence_len > model_config.max_sequence_length
            ):
                LOG.info(f"elif hasattr(model_config, "'max_sequence_length'") and model_config.max_sequence_length and cfg.sequence_len > model_config.max_sequence_length:")
                model_config.max_sequence_length = cfg.sequence_len
                LOG.warning(f"increasing context length to {cfg.sequence_len}")
            if cfg.gptq:
                model = AutoModelForCausalLM.from_pretrained(
                    base_model,
                    config=model_config,
                    trust_remote_code=cfg.trust_remote_code or False,
                    **model_kwargs,
                )
            else:
                LOG.info(f"else: model = AutoModelForCausalLM.from_pretrained")
                model = AutoModelForCausalLM.from_pretrained(
                    base_model,
                    config=model_config,
                    trust_remote_code=cfg.trust_remote_code or False,
                    **model_kwargs,
                )
    except Exception as err:  # pylint: disable=broad-exception-caught
        LOG.exception(err)
        raise err

    if isinstance(model, (PeftModel, PeftModelForCausalLM)):
        LOG.info(f"if isinstance(model, (PeftModel, PeftModelForCausalLM)):")
        model = model.merge_and_unload()

    embeddings_len = len(tokenizer)
    
    if (
        hasattr(model, "get_input_embeddings")
        and model.get_input_embeddings().num_embeddings < embeddings_len
    ):
        LOG.info(f"if문 hasattr(model, "'get_input_embeddings'") and model.get_input_embeddings().num_embeddings < embeddings_len:")
        model.resize_token_embeddings(embeddings_len)
    else:
        model.tie_weights()

    if (
        hasattr(model, "config")
        and hasattr(model.config, "max_position_embeddings")
        and model.config.max_position_embeddings
        and cfg.sequence_len > model.config.max_position_embeddings
    ):
        model.config.max_position_embeddings = cfg.sequence_len

    if (
        hasattr(model, "config")
        and hasattr(model.config, "bos_token_id")
        and model.config.bos_token_id
        and model.config.bos_token_id != tokenizer.bos_token_id
    ):
        LOG.info(f"if문 hasattr(model, 'config') and hasattr(model.config, 'bos_token_id') and model.config.bos_token_id and model.config.bos_token_id != tokenizer.bos_token_id:")
        model.config.bos_token_id = tokenizer.bos_token_id

    if (
        hasattr(model, "config")
        and hasattr(model.config, "eos_token_id")
        and model.config.eos_token_id
        and model.config.eos_token_id != tokenizer.eos_token_id
    ):
        LOG.info(f"if문 hasattr(model, "'config'") and hasattr(model.config, "'eos_token_id'") and model.config.eos_token_id and model.config.eos_token_id != tokenizer.eos_token_id:")
        model.config.eos_token_id = tokenizer.eos_token_id

    if hasattr(model, "device") and model.device.type in ("cuda", "mps"):
        LOG.info(f"if문 hasattr(model, "'device'") and model.device.type in ('cuda', 'mps'):")
        log_gpu_memory_usage(LOG, "after model load", model.device)

    # make sure these are fp32 per Ramesh et al. (2021)
    embedding_modules = get_linear_embedding_layers(cfg.model_config_type)
    LOG.info(f"embedding_modules: {embedding_modules}")
    if not cfg.fsdp:
        # FSDP doesn't like mixed Float and BFloat16
        for name, module in model.named_modules():
            if "norm" in name or name.endswith(".gate"):
                module.to(torch.float32)
            if model_config.model_type == "btlm":
                # don't upcast lm_head for btlm
                continue
            if any(m in name for m in embedding_modules):
                if hasattr(module, "weight"):
                    module.to(torch.float32)

    needs_fa2_dtype = cfg.adapter or cfg.fsdp
    skip_prepare_model_for_kbit_training = False
    
    if cfg.adapter in ["lora", "qlora"]:
        if cfg.gradient_checkpointing:
            LOG.info("if cfg.gradient_checkpointing:")
            model.gradient_checkpointing_enable()
        if (
            cfg.load_in_8bit or cfg.load_in_4bit
        ) and not skip_prepare_model_for_kbit_training:
            LOG.info("converting PEFT model w/ prepare_model_for_kbit_training: in 8it: {cfg.load_in_8bit}, in 4it: {cfg.load_in_4bit}")
            model = prepare_model_for_kbit_training(
                model, use_gradient_checkpointing=cfg.gradient_checkpointing
            )
        needs_fa2_dtype = True

    # LlamaRMSNorm layers are in fp32 after kbit_training or full finetune, so we need to
    # convert them back to fp16/bf16 for flash-attn compatibility.
    if needs_fa2_dtype or cfg.flash_attention:
        for name, module in model.named_modules():
            if "norm" in name:
                module.to(cfg.torch_dtype)
            if any(m in name for m in embedding_modules):
                if hasattr(module, "weight"):
                    module.to(cfg.torch_dtype)

    lora_config = None
    if not reference_model or cfg.lora_model_dir:
        # if we're not loading the reference model, then we're loading the model for training
        # then the dpo trainer doesn't want the peft model loaded over it, it just wants the lora/peft config
        model, lora_config = load_adapter(model, cfg, cfg.adapter)

    if torch.cuda.device_count() > 1 and int(os.getenv("WORLD_SIZE", "1")) == 1:
        LOG.info(f"if torch.cuda.device_count() > 1 and int(os.getenv('WORLD_SIZE', '1')) == 1: {int(os.getenv('WORLD_SIZE', '1'))}")
        setattr(model, "is_parallelizable", True)
        setattr(model, "model_parallel", True)

    requires_grad = []
    for name, param in model.named_parameters(recurse=True):
        if param.requires_grad:
            # LOG.info(f"if param.requires_grad: {name}: {param.requires_grad}")
            requires_grad.append(f"{name}: {param.requires_grad}")
    if len(requires_grad) == 0:
        LOG.warning("if len(requires_grad) == 0:")
    if hasattr(model, "config"):
        LOG.info(f"if hasattr(model, 'config'): {model.config}")
        model.config.use_cache = False

    if cfg.adapter is not None:
        log_gpu_memory_usage(LOG, "after adapters", model.device)

    # TODO resume_from_checkpoint handling
    return model, lora_config

In [15]:
tokenizer = load_tokenizer(cfg)

In [16]:
model, _ = load_model(cfg, tokenizer, inference=True)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it]


AttributeError: 'torch.dtype' object has no attribute 'itemsize'

In [ ]:
model = model.to(cfg.device, dtype=cfg.torch_dtype)

In [15]:
# prompt = """### sytem:
# Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

# [INST]### Instruction:
# 아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘

# ### Input:##API 호출 정의 
# -apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user's looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it's less than 300 million, indicate it as '<3'.(format: %d)"}, "query": "Generate a standalone question which is based on the 'Q' plus the chat History, User, AI. Include date information in this query, if it has present in the contents."}


# **날짜데이터 
# 오늘: Wednesday, December 20, 2023
# 내일: Thursday, December 21, 2023
# 모레: Friday, December 22, 2023
# 이번주: Wednesday, December 20, 2023 ~ Sunday, December 24, 2023
# 다음주: Monday, December 25, 2023 ~ Sunday, December 31, 2023

# **이전 대화 데이터
# User는 12월 22일부터 말까지 하는 뮤지컬공연이랑 클래식 공연 정보 알려줘.
# AI는 12월 22일부터 12월 말까지 서울에서 진행되는 뮤지컬과 클래식 공연 정보를 안내해드리겠습니다.. 사용자는 이 공연 장소의 주차장이 있어? AI는 네, 세종M씨어터와 세종체임버홀에서 진행되는 공연 장소인 세종문화회관에는 주차장이 마련되어 있습니다. 세종문화회관 공영주차장의 주차요금 및 요금할인 조건에 대한 정보는 세종문화회관 공영주차장 주차요금 및 요금할인 조건 또는 세종로 공영주차장 이용안내를 확인하시면 됩니다. 관람객 할인이 적용되는 경우도 있으니, 공연 관람 전에 해당 정보를 미리 확인하시는 것이 좋겠습니다.

# ### ouput: """

prompt = """### sytem:
Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘

### Input:##API 호출 정의 
-apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user's looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it's less than 300 million, indicate it as '<3'.(format: %d)"}, "query": "Generate a standalone question which is based on the 'Q' plus the chat History, User, AI. Include date information in this query, if it has present in the contents."}


**날짜데이터 
오늘: Wednesday, December 20, 2023
내일: Thursday, December 21, 2023
모레: Friday, December 22, 2023
이번주: Wednesday, December 20, 2023 ~ Sunday, December 24, 2023
다음주: Monday, December 25, 2023 ~ Sunday, December 31, 2023

**이전 대화 데이터
User는 12월 22일부터 말까지 하는 뮤지컬공연이랑 클래식 공연 정보 알려줘.
AI는 12월 22일부터 12월 말까지 서울에서 진행되는 뮤지컬과 클래식 공연 정보를 안내해드리겠습니다.. 사용자는 이 공연 장소의 주차장이 있어? AI는 네, 세종M씨어터와 세종체임버홀에서 진행되는 공연 장소인 세종문화회관에는 주차장이 마련되어 있습니다. 세종문화회관 공영주차장의 주차요금 및 요금할인 조건에 대한 정보는 세종문화회관 공영주차장 주차요금 및 요금할인 조건 또는 세종로 공영주차장 이용안내를 확인하시면 됩니다. 관람객 할인이 적용되는 경우도 있으니, 공연 관람 전에 해당 정보를 미리 확인하시는 것이 좋겠습니다.

### ouput:"""

batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=True) 

In [16]:
from transformers import GenerationConfig, TextIteratorStreamer, TextStreamer

generation_config = GenerationConfig(
                # repetition_penalty=1.1,
                max_new_tokens=200,
                temperature=0.1,
                top_p=0.95,
                top_k=200,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=True,
                use_cache=True,
                # return_dict_in_generate=True,
                # output_attentions=False,
                # output_hidden_states=False,
                # output_scores=False,
            )
streamer = TextStreamer(tokenizer)
generated = model.generate(
    inputs=batch["input_ids"].to(cfg.device),
    generation_config=generation_config,
    streamer=streamer,
)


<s>### sytem:
Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘

### Input:##API 호출 정의 
-apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user's looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it's less than 300 million, indicate it as '<3'.(format: %d)"}, "query": "Generate a standalone question which is based on the 'Q' plus the chat History, User, AI. Include date information in this query, if it ha

ouput: {"apt_analysis":[{"RequiredKeys":{"location":"동자동"},"OptionalKeys":{"price":">=2&<=3"},"query":"동자동의 2억원에서 3억원 사이의 아파트 시세"},{"RequiredKeys":{"location":"이태원"},"OptionalKeys":{"price":">=2&<=3"},"query":"이태원의 2억원에서 3억원 사이의 아파트  시세"}]}</s>


In [17]:
#mistral 은 사용 가능 

generated_ids = model.generate(
    inputs=batch["input_ids"].to(cfg.device),
    generation_config=generation_config,
)
generated_texts = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

# 생성된 텍스트 출력
for text in generated_texts:
    print(text)
    



# generated_ids = model.generate(
#     inputs=batch["input_ids"].to(cfg.device),
#     generation_config=generation_config,
# )
# # sequences 속성에서 토큰 ID 텐서 추출
# token_ids_tensor = generated_ids.sequences

# # 텐서를 리스트로 변환 (batch 처리를 고려하여)
# token_ids_list = token_ids_tensor.tolist()

# # 각 시퀀스에 대해 디코딩 수행
# generated_texts = [tokenizer.decode(g, skip_special_tokens=True) for g in token_ids_list]

# # 생성된 텍스트 출력
# for text in generated_texts:
#     print(text)


### sytem:
Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘

### Input:##API 호출 정의 
-apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user's looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it's less than 300 million, indicate it as '<3'.(format: %d)"}, "query": "Generate a standalone question which is based on the 'Q' plus the chat History, User, AI. Include date information in this query, if it has p

In [26]:
generated_ids

GenerateDecoderOnlyOutput(sequences=tensor([[    1,   774,   268,  4475,   366, 28747,    13, 18066, 29148, 28705,
         31359, 29700, 28705, 30151,   239,   163,   139, 29136, 29833, 11232,
         28705, 29969, 29015, 29665, 29200, 28705, 31273, 29424, 29136, 29827,
         15985, 28705, 31647, 29901, 29189, 28705, 29136, 29721, 29517, 28723,
         28705, 30933, 29175, 28705, 29315, 29424, 29294, 28705, 31959, 29710,
         28732, 18066, 28731, 29148, 28705, 29991, 29236, 30112, 29583, 29143,
          7086, 28705, 29730, 29858, 29189, 28705, 29744, 29282, 17750, 28705,
         30498, 30355, 29187, 28705, 29779, 29841, 29731, 29897, 28705, 30075,
         29844, 29200, 28705, 29693, 29465, 29136, 29175, 28705, 29324, 30010,
         29161, 30364, 29015, 30263, 28723,    13,    13, 28792, 16289, 28793,
         27332,  3133,  3112, 28747,    13, 29536, 29806, 29404, 28705, 29236,
         29721, 29200, 28705, 30217, 31585, 29136, 29511, 28705,   239,   142,
           185, 

In [59]:
generated_texts

['### sytem:\nInstruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.\n\n[INST]### Instruction:\n아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘\n\n### Input:##API 호출 정의 \n-apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user\'s looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it\'s less than 300 million, indicate it as \'<3\'.(format: %d)"}, "query": "Generate a standalone question which is based on the \'Q\' plus the chat History, User, AI. Include date information in this que

In [57]:
tokenizer.decode(generated["sequences"].cpu().tolist()[0])

IndexError: too many indices for tensor of dimension 2

In [60]:
generated = model.generate(
    inputs=batch["input_ids"].to(cfg.device),
    generation_config=generation_config,
    streamer=streamer,
)

<s>### sytem:
Instruction에 따라 적절하게 Input 데이터를 활용하여 Output 답변을 하세요. 너는 사용자 질문(Instruction)에 실시간으로 API 호출을 위한 Json 형식의 구조화된 결과를 생성하는 인공지능이야.

[INST]### Instruction:
아파트 시세를 분석하고 싶어 특히 동자동 ! 2억원에서 3억원 사이로는?아. 이태원도 같은 가격대로 같이 알려줘

### Input:##API 호출 정의 
-apt_analysis: {"Name": "서울 아파트 상권분석", "Description": "서울 아파트 상권 분석 서비스 API를 사용합니다. 서울시 행정동별 아파트의 통계 정보를 제공합니다. It analyzes commercial districts based on statistical information. Provides details such as number of apartment complexes, number of units by apartment square footage, number of units by apartment price, average apartment square footage, and market value.", "RequiredKeys": {"location": "The name of the location user's looking for"}, "OptionalKeys": {"price": "The range of apartment prices you want to find. For example, if it's less than 300 million, indicate it as '<3'.(format: %d)"}, "query": "Generate a standalone question which is based on the 'Q' plus the chat History, User, AI. Include date information in this query, if it ha

 {"apt_analysis":[{"RequiredKeys":{"location":"동자동"},"OptionalKeys":{"price":">=2&<=3"},"query":"동자동의 2억원에서 3억원 사이의 아파트 시세"},{"RequiredKeys":{"location":"이태원"},"OptionalKeys":{"price":">=2&<=3"},"query":"이태원의 2억원에서 3억원 사이의 아파트  시세"}]}</s>


In [62]:
print("=" * 40)
print(tokenizer.decode(generated["sequences"].cpu().tolist()[0]))


IndexError: too many indices for tensor of dimension 2